# A Transport Phenomena Case Study: Heat Transfer in a Pipe

**Objective:** This lesson will walk through a classic transport phenomena problem that combines fluid mechanics, heat transfer, and an overall energy balance. We will simulate the temperature profile of a fluid as it heats up while flowing through a hot-walled pipe.

**Learning Goals:**
1.  Understand the interplay between Momentum Transport (fluid flow) and Heat Transport.
2.  Apply key dimensionless numbers: Reynolds ($Re$), Prandtl ($Pr$), and Nusselt ($Nu$).
3.  Calculate the convective heat transfer coefficient ($h$) using empirical correlations.
4.  Develop a dynamic model based on a differential energy balance along the pipe.
5.  Solve the resulting Ordinary Differential Equation (ODE) to find the temperature profile and validate it against the analytical solution.

## The Problem: The Double-Pipe Heat Exchanger

Imagine a simple heat exchanger where a cold fluid flows through the inner pipe, while steam condenses on the outside of the pipe, keeping the outer wall at a constant, high temperature. We want to determine the temperature of the cold fluid as it travels along the length of the pipe.

![Pipe Flow Diagram](https://cdn.thomasnet.com/insights-images/embedded-images/e1606558-2a39-43dd-891b-3e450877b1e8/cd5ad77a-b29f-4eb8-a526-8844595493ae/FullHD/Heat-Transfer-in-a-Double-Pipe-Heat-Exchanger.jpg)

To solve this, we need to answer a key question: **How quickly does heat get from the hot wall into the moving fluid?** The answer lies in the convective heat transfer coefficient, $h$.

## Part 1: Momentum Transport - Characterizing the Flow

First, we need to understand the nature of the fluid flow. Is it smooth and orderly (laminar) or chaotic and well-mixed (turbulent)? This is determined by the **Reynolds Number ($Re$)**, which is the ratio of inertial forces to viscous forces.
$$ Re = \frac{\rho v D}{\mu} $$
where:
*   $\rho$ = fluid density (kg/m³)
*   $v$ = average fluid velocity (m/s)
*   $D$ = pipe diameter (m)
*   $\mu$ = fluid viscosity (Pa·s)

For pipe flow, the general rules are:
*   $Re < 2100$: Laminar Flow
*   $Re > 4000$: Turbulent Flow

The flow regime is critical because it dramatically affects the heat transfer. Turbulent flow is a much more effective way to transfer heat.

## Part 2: Heat Transport - Quantifying Heat Transfer

To find the heat transfer coefficient, $h$, we use other dimensionless numbers. 

**1. The Prandtl Number ($Pr$)** is a fluid property that relates momentum diffusivity to thermal diffusivity.
$$ Pr = \frac{C_p \mu}{k_f} $$
where $C_p$ is the fluid heat capacity and $k_f$ is the fluid thermal conductivity.

**2. The Nusselt Number ($Nu$)** is the ratio of convective heat transfer to conductive heat transfer. It's the dimensionless heat transfer coefficient. 
$$ Nu = \frac{h D}{k_f} $$

The key is that there are well-established empirical correlations that relate the Nusselt number to the Reynolds and Prandtl numbers. For turbulent flow in a pipe, a common one is the **Gnielinski Correlation**:
$$ Nu_D = \frac{(f_D/8)(Re_D - 1000)Pr}{1 + 12.7(f_D/8)^{1/2}(Pr^{2/3}-1)} $$
where $f_D$ is the Darcy friction factor, another key parameter from momentum transport.

## Part 3: The Overall Model - Differential Energy Balance

Consider a small, differential slice of the pipe of length $dz$. The rate of heat entering this slice from the wall must equal the rate at which the fluid's energy increases.

**Heat in from wall ($dQ$):**
$$ dQ = h \cdot (\text{Surface Area}) \cdot (T_{wall} - T_{fluid}) = h \cdot (\pi D \, dz) \cdot (T_{wall} - T) $$

**Energy increase of fluid (rate of change of enthalpy):**
$$ dQ = \dot{m} C_p \, dT $$
where $\dot{m}$ is the mass flow rate.

Setting these equal:
$$ \dot{m} C_p \, dT = h \pi D (T_{wall} - T) \, dz $$

Rearranging gives our final ODE for the temperature $T$ as a function of position $z$:
$$ \frac{dT}{dz} = \frac{h \pi D}{\dot{m} C_p} (T_{wall} - T) $$

## Part 4: Python Implementation and Simulation

In [ ]:
# Import necessary libraries
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# --- Define Physical Properties for Water at ~40 C ---
rho = 992.2      # Density (kg/m^3)
mu = 0.653e-3    # Viscosity (Pa.s)
Cp = 4179        # Heat Capacity (J/kg.K)
kf = 0.631       # Thermal Conductivity (W/m.K)

# --- Define Geometry and Operating Conditions ---
D = 0.05         # Pipe Diameter (m)
L = 20.0         # Pipe Length (m)
mass_flow = 1.0  # Mass flow rate (kg/s)
T_inlet = 20.0     # Inlet fluid temperature (C)
T_wall = 100.0     # Pipe wall temperature (C)

print("Parameters are defined.")

In [ ]:
# --- Transport Phenomena Calculations ---

# 1. Calculate velocity and Reynolds number
Area = np.pi * (D**2) / 4
velocity = mass_flow / (rho * Area)
Re = (rho * velocity * D) / mu

# 2. Calculate Darcy Friction Factor (using Churchill Equation, valid for all regimes)
A_f = (2.457 * np.log(1 / ((7/Re)**0.9)))**16
B_f = (37530 / Re)**16
f_D = 8 * ((8/Re)**12 + 1 / (A_f + B_f)**1.5)**(1/12)

# 3. Calculate Prandtl and Nusselt numbers
Pr = (Cp * mu) / kf
Nu = ((f_D/8)*(Re-1000)*Pr) / (1 + 12.7*(f_D/8)**0.5 * (Pr**(2/3)-1)) # Gnielinski

# 4. Calculate the all-important heat transfer coefficient, h
h = (Nu * kf) / D

print(f"Calculated Reynolds Number: {Re:.0f} (Turbulent Flow)")
print(f"Calculated Heat Transfer Coefficient (h): {h:.2f} W/m^2.K")

In [ ]:
# --- Define and Solve the ODE Model ---

def pipe_heat_model(z, T):
    """
    Defines the ODE for fluid temperature along the pipe.
    Args:
        z (float): Position along the pipe (m).
        T (float): Fluid temperature at position z (C).
    """
    # Note: For simplicity, we assume h is constant along the pipe.
    # In reality, fluid properties (rho, mu, etc.) change with T, making h(z).
    # This is a common and valid simplification for many problems.
    
    dTdz = (h * np.pi * D) / (mass_flow * Cp) * (T_wall - T)
    return dTdz

# Simulation parameters
z_span = (0, L)              # Solve from z=0 to z=L
T_initial = [T_inlet]        # Initial condition (temperature at z=0)

# Call the solver
solution = solve_ivp(pipe_heat_model, z_span, T_initial, dense_output=True)

# Generate points for a smooth plot
z_plot = np.linspace(z_span[0], z_span[1], 100)
T_plot = solution.sol(z_plot)[0]

print("ODE Simulation complete.")

## Part 5: Analysis and Validation

Let's plot our simulation result. A key feature should be that the fluid temperature rises and asymptotically approaches the wall temperature.

In [ ]:
# --- Validation with Analytical Solution ---
# This ODE has a known analytical solution, which is great for checking our work.
exp_term = np.exp(-(h * np.pi * D * z_plot) / (mass_flow * Cp))
T_analytical = T_wall - (T_wall - T_inlet) * exp_term

# --- Plotting ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(12, 7))

plt.plot(z_plot, T_plot, 'b-', linewidth=3, label='Numerical Simulation (ODE Solver)')
plt.plot(z_plot, T_analytical, 'r--', linewidth=2, label='Analytical Solution')
plt.axhline(y=T_wall, color='black', linestyle=':', label=f'Wall Temperature ({T_wall} C)')

plt.title('Fluid Temperature Profile Along Pipe', fontsize=18, weight='bold')
plt.xlabel('Position along pipe, z (m)', fontsize=14)
plt.ylabel('Fluid Temperature (C)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

print(f"The final outlet temperature is {T_plot[-1]:.2f} C.")
print("The numerical simulation perfectly matches the analytical solution, validating our model!")

## Part 6: Be the Transport Phenomena Engineer!

Now, use the simulation to explore how design changes affect performance. Modify the **Geometry and Operating Conditions** in **Part 4** and re-run the cells.

#### Challenge 1: Effect of Flow Rate
What happens if you **double the mass flow rate** to `2.0` kg/s? 
*Before you run, predict*: Will the Reynolds number increase or decrease? How will that affect the heat transfer coefficient, $h$? Will the final outlet temperature be higher or lower than the base case? Why?

#### Challenge 2: Effect of Pipe Diameter
Reset the flow rate. Now, **reduce the pipe diameter** `D` to `0.025` m. 
*Predict*: How does a smaller pipe affect the fluid velocity and the Reynolds number? What will be the net effect on the outlet temperature?

#### Challenge 3: Change of Fluid
This is a bigger challenge. Imagine you are now heating a viscous oil instead of water. Oil has a much higher viscosity (e.g., `mu = 0.1` Pa.s) and a lower heat capacity and thermal conductivity. Make these changes to the physical properties.
*Predict*: Will the flow be laminar or turbulent? How will the much higher Prandtl number affect heat transfer? Will you need a much longer pipe to achieve the same heating?